In [1]:
import numpy as np
import math
# Set up environment
import gym
env = gym.make("CartPole-v0")
# Initial observation
observation = env.reset()

In [1]:
# q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [2]:
# Function for discretizising the env state
# The data from the env is continious and needs to be discretized befored being passed to the agent
def discretize_state(self, observation):
        discretized = list()
        for i in range(len(obs)):
            scaling = ((obs[i] + abs(self.lower_bounds[i]))
                      / (self.upper_bounds[i] - self.lower_bounds[i]))
            new_obs = int(round((self.buckets[i] - 1) * scaling))
            new_obs = min(self.buckets[i] - 1, max(0, new_obs))
            discretized.append(new_obs)
        return tuple(discretized)

In [4]:
# Define agent class
class agent:
    # Constructor
    def __init__(self, buckets=(3, 3, 6, 6), num_episodes=500, min_learning_rate=0.1, min_epsilon=0.1, discount=1.0, decay=25):
        
        self.buckets = buckets
        self.num_episodes = num_episodes
        self.min_learning_rate = min_learning_rate
        self.min_epsilon = min_epsilon
        self.discount = discount
        self.decay = decay
        
        
        self.q_table = np.zeros(self.buckets + (env.action_space.n,))
        
        # Upper and lower limits
        self.upper_limits = [env.observation_space.high[0], 0.5, env.observation_space.high[2], math.radians(50)/1.]
        self.lower_limits = [env.observation_space.low[0], -0.5, env.observation_space.low[2], -math.radians(50)/1.]
        
        self.steps = np.zeros(self.num_episodes)
    
    def choose_action(self, state):
        
        if (np.random.random() < self.epsilon):
            return env.action_space.sample()
        else:
            return np.argmax(self.q_table[state])
        
    
    

In [14]:

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

for _ in range(1000):
  env.render()
  action = env.action_space.sample() # your agent here (this takes random actions)
  observation, reward, done, info = env.step(action)

  if done:
    observation = env.reset()
env.close()

Action Space Discrete(2)
State Space Box(-3.4028234663852886e+38, 3.4028234663852886e+38, (4,), float32)
